# 02 — Data Preprocessing & Feature Engineering

This notebook prepares the **Diabetes Hospital Stay Prediction** dataset for modeling.  
We perform data cleaning, outlier handling, encoding, and feature construction  
to obtain a machine-learning–ready dataset for regression analysis.

---

### 🎯 Objectives
1. Load raw data and verify consistency  
2. Remove irrelevant or duplicate identifiers  
3. Handle missing and inconsistent values  
4. Engineer clinically meaningful features  
5. Encode categorical variables  
6. Save the processed dataset for modeling

> **Outcome:** A clean, fully numeric dataset ready for regression models in `03_model_training.ipynb`.



## 1️⃣ Load Raw Dataset

We begin by loading the raw diabetes dataset and inspecting its structure.  
This ensures the same data consistency seen during exploratory analysis.


In [16]:
# Imports and paths
import os
import numpy as np
import pandas as pd

RAW_PATH = "../data/raw/diabetic_data.csv"   
PROC_DIR = "../data/processed"
os.makedirs(PROC_DIR, exist_ok=True)

df = pd.read_csv(RAW_PATH)
print("Loaded:", df.shape)
df.head()


Loaded: (101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [17]:
# Replace '?' with np.nan
df = df.replace('?', np.nan)

# Fill specific categorical columns with known labels
fill_map = {
    "A1Cresult": "None",
    "max_glu_serum": "None",
    "weight": "Unknown",
    "race": "Unknown"
}

for col, fillval in fill_map.items():
    if col in df.columns:
        df[col] = df[col].fillna(fillval)

# Quick check for these columns
df[["A1Cresult", "max_glu_serum", "weight", "race"]].head()


,A1Cresult,max_glu_serum,weight,race
0,None,None,Unknown,Caucasian
1,None,None,Unknown,Caucasian
2,None,None,Unknown,AfricanAmerican
3,None,None,Unknown,Caucasian
4,None,None,Unknown,Caucasian


## 2️⃣ Remove Non-Predictive Identifiers

Columns like `encounter_id` and `patient_nbr` are unique per admission and do not contain predictive information.  
They are removed to avoid data leakage and reduce dimensionality.


In [18]:
drop_cols = [c for c in ["encounter_id", "patient_nbr"] if c in df.columns]
df = df.drop(columns=drop_cols)
print("Dropped columns:", drop_cols)
print("New shape:", df.shape)


Dropped columns: ['encounter_id', 'patient_nbr']
New shape: (101766, 48)


## 3️⃣ Convert Age Ranges to Numeric Midpoints

The age variable is encoded as ranges (e.g., `[70-80)`),  
which are converted into numeric midpoints for modeling continuity.



In [19]:
def age_bin_to_mid(x):
    if isinstance(x, str) and "-" in x:
        try:
            a, b = x.strip("[]()").split("-")
            return (float(a) + float(b)) / 2
        except:
            return np.nan
    return np.nan

if "age" in df.columns:
    df["age_mid"] = df["age"].apply(age_bin_to_mid)
    print(df[["age", "age_mid"]].head(10))
else:
    print("Column 'age' not found.")


        age  age_mid
0    [0-10)      5.0
1   [10-20)     15.0
2   [20-30)     25.0
3   [30-40)     35.0
4   [40-50)     45.0
5   [50-60)     55.0
6   [60-70)     65.0
7   [70-80)     75.0
8   [80-90)     85.0
9  [90-100)     95.0


## 4️⃣ Feature Engineering: Clinical Intensity & Visit History

We create additional features representing:
- **Medication / Lab / Procedure intensity:** normalized per hospital day  
- **Prior visit activity:** sum of outpatient, emergency, and inpatient visits  
- **Binary flag (`had_prior_visit`):** whether the patient had prior admissions

> These derived variables enhance the model’s ability to capture patient complexity.



In [20]:
def safe_div(a, b):
    """Avoid division by zero"""
    return np.where(b == 0, 0, a / b)

req_cols = ["time_in_hospital","num_medications","num_lab_procedures","num_procedures",
            "number_outpatient","number_emergency","number_inpatient"]

missing_req = [c for c in req_cols if c not in df.columns]
if missing_req:
    print("Missing required columns:", missing_req)
else:
    df["meds_per_day"] = safe_div(df["num_medications"].values, df["time_in_hospital"].values)
    df["labs_per_day"] = safe_div(df["num_lab_procedures"].values, df["time_in_hospital"].values)
    df["procs_per_day"] = safe_div(df["num_procedures"].values, df["time_in_hospital"].values)

    df["prior_visits"] = df["number_outpatient"] + df["number_emergency"] + df["number_inpatient"]
    df["had_prior_visit"] = (df["prior_visits"] > 0).astype(int)

df[["meds_per_day","labs_per_day","procs_per_day","prior_visits","had_prior_visit"]].head()


,meds_per_day,labs_per_day,procs_per_day,prior_visits,had_prior_visit
0,1.0,41.000000,0.0,0,0
1,6.0,19.666667,0.0,0,0
2,6.5,5.500000,2.5,3,1
3,8.0,22.000000,0.5,0,0
4,8.0,51.000000,0.0,0,0


## 5️⃣ High-Cardinality Category Reduction

Several categorical features (especially diagnosis codes and medical specialty) contain a very large number of distinct values.  
To reduce noise and improve model stability, we group rare categories under a common `"Other"` label:

- `diag_1`, `diag_2`, `diag_3`: keep the **top 50** most frequent categories, map all remaining values to `Other`.  
- `medical_specialty`: keep the **top 30** most frequent categories, map all remaining values to `Other`.  

This step:
- Reduces the dimensionality after one-hot encoding  
- Helps prevent overfitting on very rare categories  
- Speeds up training without losing the main clinical signal


In [21]:
def clip_top_k(series, k=50, other_label="Other"):
    top = series.value_counts().nlargest(k).index
    return series.where(series.isin(top), other_label)

for diag_col in ["diag_1","diag_2","diag_3"]:
    if diag_col in df.columns:
        df[diag_col] = clip_top_k(df[diag_col].astype(str), k=50)

if "medical_specialty" in df.columns:
    df["medical_specialty"] = clip_top_k(df["medical_specialty"].astype(str), k=30)

df[["diag_1","diag_2","diag_3","medical_specialty"]].head()


,diag_1,diag_2,diag_3,medical_specialty
0,Other,nan,nan,Pediatrics-Endocrinology
1,276,250.01,Other,nan
2,Other,250,Other,nan
3,8,Other,403,nan
4,Other,Other,250,nan


## 6️⃣ Normalize Missing Markers in Categorical Features

After clipping high-cardinality categories, we ensure that all missing or inconsistent markers  
(e.g. `NaN`, `"nan"`, `None`) are standardized to a single `"Unknown"` label  
for key categorical features such as diagnoses and medical specialty.

> This prevents unexpected category fragmentation during one-hot encoding.


In [22]:
# Normalize missing markers for clipped categoricals
fix_cats = ["diag_1", "diag_2", "diag_3", "medical_specialty"]

for col in fix_cats:
    if col in df.columns:
        df[col] = df[col].replace({np.nan: "Unknown", "nan": "Unknown", "NaN": "Unknown", None: "Unknown"})

# Quick check
for col in fix_cats:
    if col in df.columns:
        print(col, "-> unique sample:", df[col].unique()[:8])


diag_1 -> unique sample: ['Other' '276' '8' '414' '428' '434' '250.7' '518']
diag_2 -> unique sample: ['Unknown' '250.01' '250' 'Other' '411' '427' '403' '998']
diag_3 -> unique sample: ['Unknown' 'Other' '403' '250' 'V45' '486' '197' '250.6']
medical_specialty -> unique sample: ['Pediatrics-Endocrinology' 'Unknown' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedics'
 'Gastroenterology']


## 7️⃣ Outlier Handling via Winsorization (1%–99%)

To reduce the influence of extreme encounter counts,  
we clip heavy-tailed variables (`number_outpatient`, `number_emergency`, `number_inpatient`)  
between their 1st and 99th percentiles.

> This preserves the overall distribution while mitigating the impact of rare, extreme values.


In [23]:
# Clip heavy-tailed counts at [1%, 99%] to reduce extreme outliers
WINSORIZE = True
winsor_cols = [c for c in ["number_outpatient","number_emergency","number_inpatient"] if c in df.columns]

if WINSORIZE and winsor_cols:
    for col in winsor_cols:
        low, high = df[col].quantile([0.01, 0.99])
        df[col] = df[col].clip(lower=low, upper=high)
    print("Winsorized:", winsor_cols)
    print(df[winsor_cols].describe(percentiles=[.01,.5,.99]))
else:
    print("Winsorization skipped or columns not present.")


Winsorized: ['number_outpatient', 'number_emergency', 'number_inpatient']
       number_outpatient  number_emergency  number_inpatient
count      101766.000000     101766.000000     101766.000000
mean            0.329599          0.164623          0.615707
std             0.909908          0.525132          1.136751
min             0.000000          0.000000          0.000000
1%              0.000000          0.000000          0.000000
50%             0.000000          0.000000          0.000000
99%             5.000000          3.000000          6.000000
max             5.000000          3.000000          6.000000


## 8️⃣ Prepare Categorical Features for Encoding

We explicitly collect the main categorical features to be one-hot encoded, including  
demographics, diagnoses, lab-related flags, and admission/discharge codes.

- ID-like categorical variables (`admission_type_id`, `discharge_disposition_id`, `admission_source_id`)  
  are cast to `str` to avoid being treated as numeric.
- The original `age` range column is dropped after creating the numeric `age_mid` representation.

> This step ensures a clean separation between numeric and categorical inputs before encoding.


In [24]:
# Prepare categorical columns for one-hot encoding
TARGET = "time_in_hospital"

cat_cols = []
for c in ["race","gender","A1Cresult","max_glu_serum","medical_specialty",
          "diag_1","diag_2","diag_3","weight",
          "admission_type_id","discharge_disposition_id","admission_source_id"]:
    if c in df.columns:
        cat_cols.append(c)

# Ensure id-coded categoricals are strings
for c in ["admission_type_id","discharge_disposition_id","admission_source_id"]:
    if c in cat_cols:
        df[c] = df[c].astype(str)

# Drop original age after creating numeric midpoint
if "age_mid" in df.columns and "age" in df.columns:
    df = df.drop(columns=["age"])

print("Categoricals to encode:", len(cat_cols), "| Example:", cat_cols[:6])
print("Current shape:", df.shape)


Categoricals to encode: 12 | Example: ['race', 'gender', 'A1Cresult', 'max_glu_serum', 'medical_specialty', 'diag_1']
Current shape: (101766, 53)


## 9️⃣ One-Hot Encoding & Final Missing-Value Cleanup

We apply one-hot encoding to the selected categorical variables using `drop_first=True`  
to reduce collinearity between dummy variables.

Additionally, if `payer_code` is present, any remaining missing entries are mapped to `"Unknown"`,  
and a final sanity check confirms that:
- The target column is present  
- No missing values remain in the feature matrix


In [25]:
# One-hot encode with drop_first to reduce collinearity
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
print("Encoded shape:", df_encoded.shape)

# Quick sanity
print("Total NaNs after encoding:", int(df_encoded.isna().sum().sum()))
print("Has target:", TARGET in df_encoded.columns)
df_encoded.head(3)


Encoded shape: (101766, 291)
Total NaNs after encoding: 40256
Has target: True


,time_in_hospital,payer_code,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,...,admission_source_id_20,admission_source_id_22,admission_source_id_25,admission_source_id_3,admission_source_id_4,admission_source_id_5,admission_source_id_6,admission_source_id_7,admission_source_id_8,admission_source_id_9
0,1,NaN,41,0,1,0,0,0,1,No,...,False,False,False,False,False,False,False,False,False,False
1,3,NaN,59,0,18,0,0,0,9,No,...,False,False,False,False,False,False,False,True,False,False
2,2,NaN,11,5,13,2,0,1,6,No,...,False,False,False,False,False,False,False,True,False,False


In [26]:
if "payer_code" in df_encoded.columns:
    df_encoded["payer_code"] = df_encoded["payer_code"].fillna("Unknown")


In [27]:
# Quick sanity
print("Total NaNs after encoding:", int(df_encoded.isna().sum().sum()))
print("Has target:", TARGET in df_encoded.columns)
df_encoded.head(3)

Total NaNs after encoding: 0
Has target: True


,time_in_hospital,payer_code,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,...,admission_source_id_20,admission_source_id_22,admission_source_id_25,admission_source_id_3,admission_source_id_4,admission_source_id_5,admission_source_id_6,admission_source_id_7,admission_source_id_8,admission_source_id_9
0,1,Unknown,41,0,1,0,0,0,1,No,...,False,False,False,False,False,False,False,False,False,False
1,3,Unknown,59,0,18,0,0,0,9,No,...,False,False,False,False,False,False,False,True,False,False
2,2,Unknown,11,5,13,2,0,1,6,No,...,False,False,False,False,False,False,False,True,False,False


## 🔟 Save Processed Dataset

The fully cleaned and encoded dataset is saved as `cleaned_data.csv`  
under the `data/processed/` directory.

> This file will be used as the main input for model training in `03_model_training.ipynb`.


In [28]:
import os
PROC_DIR = "../data/processed"
os.makedirs(PROC_DIR, exist_ok=True)
OUT_PATH = os.path.join(PROC_DIR, "cleaned_data.csv")

df_encoded.to_csv(OUT_PATH, index=False)
print("Saved to:", OUT_PATH, "| Shape:", df_encoded.shape)


Saved to: ../data/processed\cleaned_data.csv | Shape: (101766, 291)
